In [2]:
import pandas as pd
import pickle
from Bio import SeqIO
from sklearn.metrics import classification_report
import numpy as np

In [3]:
PATH_SAMPLE_IDS = '/home/parazit/ml_virus_host/v2.0/sample_ids/genomes_fragments_connected/' # путь к файлам, содержащим AC геномов тренировочной, валидайионной и тестовой выборок
PATH_DATA = '/home/parazit/ml_virus_host/v2.0/data/' # путь к fasta-файлам (геномы и фрагменты), таблицам с аннотцией
PATH_BLASTN = "/home/parazit/ml_virus_host/v2.0/v3.0/baseline/blastn/" # рабочая папка 

Таблицы с аннотацией геномов и фрагментов

In [4]:
meta_df_genomes = pd.read_csv(PATH_DATA+"data_table.tsv", sep="\t", index_col = 0)
meta_df_800 = pd.read_csv(PATH_DATA+"data_table_800.tsv", sep="\t", index_col = 0)
meta_df_400 = pd.read_csv(PATH_DATA+"data_table_400.tsv", sep="\t", index_col = 0)

Списки AC геномов или фрагметов тренировочной, валидационной и тестовой выборок

In [5]:
sample_ids_genomes = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_genomes.pkl", "rb"))
sample_ids_800 = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_800.pkl", "rb"))
sample_ids_400 = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_400.pkl", "rb"))

Функция весов хозяев

In [6]:
def find_and_weight_hosts(finding, df, func_type):
    
    # выбираем функцию для подсчёта веса
    if func_type:
        function = finding.pident*finding.qcovs
    else:
        function = finding.pident
    
    # создаём таблицу, в которую записываем:
    tmp = pd.DataFrame()
    tmp["findings"] = finding.sseqid.values # AC каждой находоки
    tmp["host"] = df.loc[finding.sseqid.values].host.values # хозяина каждой находоки
    tmp["weight"] = (function/function.sum()).values # вес каждой находоки
    
    tmp.fillna(0, inplace=True)
    tmp["index"] = finding.index
    
    # в случае, если вес хозяина равен 0 (обычно при qcovs*pident значение для qcovs может равняться 0),
    # то присваиваем класс "unclassified"
    tmp.loc[tmp.weight == 0, "host"] = "unclassified" 
    tmp.set_index("index", drop=True, inplace=True)

    
    return(tmp)

Функция, анализирующая выдачу blastn

In [10]:
def blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name):

    # blastn_out - выдача blastn
    # df_query - таблица с аннотацией геномов/фрагментов, по которым проводится поиск blastn
    # df_db - таблица с аннотацией геномов, по которым создаётся база бласт
    # ids_query - AC геномов/фрагментов тестовой выборки
    # weight_type - тип веса хозяев; 0 - pident, 1 - pident*qcovs
    # save_name - название файла, который сохранит данная функция
    
    y_pred, y_true, weights = [], df_query.loc[ids_query].host.values, []


    for seq_id in ids_query: # проходимся по каждому id из тестовой выборки

        try:
            match = blastn_out.loc[seq_id] # достаём все находки бласт для данного id из тестовой выборки
            
            if type(match) == type(pd.DataFrame()): # если несколько находок, то .loc выдаёт pandas.DataFrame()
                tmp = find_and_weight_hosts(match, df_db, weight_type).groupby("host").weight.sum() # функция подсчёта весов
                predicted_host = max(zip(tmp.values, tmp.index))[1] # отбираем хозяина с наибольшим весом
                weight = max(tmp.values)
                
            if type(match) == type(pd.Series(0)): # если одна находока, то .loc выдаёт pandas.Series()
                tmp = find_and_weight_hosts(pd.DataFrame(match).T, df_db, weight_type).groupby("host").weight.sum() # функция подсчёта весов
                predicted_host = max(zip(tmp.values, tmp.index))[1] # отбираем хозяина с наибольшим весом
                weight = max(tmp.values)
            
        except KeyError: # в случае, если находок бласта нет, то присваиваем класс "unclassified"
            predicted_host = "unclassified"
            weight = 1.0
        
        # сохраняем выдачу
        y_pred.append(predicted_host), weights.append(weight)
        
    print(classification_report(y_true, y_pred, zero_division=0))
    out = pd.DataFrame(zip(ids_query, y_pred, weights), columns=["AC", "host", "weights"])
    
    out.to_csv(PATH_BLASTN+"prediction/"+save_name, sep="\t", index=0)
    return

Если переменная weight_type = 1, то вес рассчитывается по формуле: pident * qcovs
Если переменная weight_type = 0, то вес рассчитывается по формуле: pident

Важно! Классы хозяев, вес которых = 0, считаются неклассифицированными
Иногда qcovs = 0 (плохие выравнивания, стараемся избавиться от них), при этом pident отличен от 0. Находки, которые определяются с помощью pident, 

In [12]:
blastn_out = pd.read_csv(PATH_BLASTN + "output/blastn_genomes.csv", sep="\t", index_col="qseqid")
df_query = meta_df_genomes
df_db = meta_df_genomes
ids_query = sample_ids_genomes[2]
weight_type = 0
save_name = "blastn_prediction_genomes.tsv"




blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.62      0.29      0.40        95
     Mammalia       0.92      0.67      0.78       160
Viridiplantae       0.97      0.40      0.56        83
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.50       338
    macro avg       0.63      0.34      0.43       338
 weighted avg       0.85      0.50      0.62       338



In [13]:
blastn_out = pd.read_csv(PATH_BLASTN + "output/blastn_800.csv", sep="\t", index_col="qseqid")
df_query = meta_df_800
df_db = meta_df_genomes
ids_query = sample_ids_800[2]
weight_type = 0
save_name = "blastn_prediction_800.tsv"




blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.47      0.11      0.18       190
     Mammalia       0.92      0.25      0.40       320
Viridiplantae       0.94      0.09      0.16       166
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.17       676
    macro avg       0.58      0.11      0.19       676
 weighted avg       0.80      0.17      0.28       676



In [15]:
blastn_out = pd.read_csv(PATH_BLASTN + "output/blastn_400.csv", sep="\t", index_col="qseqid")
df_query = meta_df_400
df_db = meta_df_genomes
ids_query = sample_ids_400[2]
weight_type = 0
save_name = "blastn_prediction_400.tsv"




blastn_analysis(blastn_out, df_query, df_db, ids_query, weight_type, save_name)

               precision    recall  f1-score   support

      Insecta       0.69      0.11      0.18       190
     Mammalia       0.98      0.12      0.22       320
Viridiplantae       1.00      0.05      0.09       166
 unclassified       0.00      0.00      0.00         0

     accuracy                           0.10       676
    macro avg       0.67      0.07      0.12       676
 weighted avg       0.90      0.10      0.18       676

